In [ ]:
import os
import pandas as pd

In [ ]:
import os
import pandas as pd


folders = ["Giáo_dục"]


document_list = []


for folder in folders:
    file_name = f"{folder.replace(' ', '_')}.csv"
    if os.path.isfile(file_name): 
        df = pd.read_csv(file_name, encoding="utf-8-sig")
        document_list.append(df)
    else:
        print(f"File không tồn tại: {file_name}")


df = pd.concat(document_list, ignore_index=True)

# Bags-of-words

In [ ]:
import re

def preprocess(text):
  
    keep_pattern = r'(?<=[a-zA-Z0-9])([.,])(?=[a-zA-Z0-9])|(?<=[a-zA-Z])\.(?=\s|,|$)'
    remove_pattern = r'[.,](?=\s|$|[^a-zA-Z0-9])'

  
    text = re.sub(keep_pattern, r'\1', text)

  
    text = re.sub(remove_pattern, ' ', text)

  
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'[/-]', ' ', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\s{2,}', ' ', text)

  
    return text.strip().split()


df['content'] = df['content'].astype(str)
documents = df['content'].map(preprocess).tolist()


dictionary = corpora.Dictionary(documents)


corpus = [dictionary.doc2bow(doc) for doc in documents]

# Xây dựng LDA

In [ ]:
num_topic = 3
lda_model = LdaModel(corpus, num_topics=num_topic, id2word=dictionary, passes=5)

## Xây dựng thống kê cho các cụm

In [ ]:
from collections import Counter

def caculate(lda_model, corpus):
    stat = []
    for doc_bow in corpus:
        topic_probs = lda_model.get_document_topics(doc_bow)
        stat_count = max(topic_probs, key=lambda x: x[1])[0]
        stat.append(stat_count)
    return stat

stat = caculate(lda_model, corpus)

topic_counts = Counter(stat)

print("\nSố lượng tài liệu thuộc mỗi chủ đề:")
for topic, count in topic_counts.items():
    print(f"Chủ đề {topic}: {count} tài liệu")

print(f"\nTổng số tài liệu: {sum(topic_counts.values())}")

## Lấy từ khóa nổi bật

In [ ]:

def print_topic_words(model, num_topics):
    
    for idx in range(num_topics):
        topic_terms = model.show_topic(idx, topn=200)
        topic_keywords = [term for term, _ in topic_terms]
        print(f"Chủ đề {idx}: {', '.join(topic_keywords)}")


print_topic_words(lda_model, num_topics)

## Phân phối chủ đề cho các tài liệu 

In [ ]:
import pandas as pd

def get_topic_distribution_for_documents(lda_model, corpus, content_list, top_n=5):
    
    result = []

    
    for idx, doc_bow in enumerate(corpus):
        
        topic_probabilities = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        
       
        sorted_topics = sorted(topic_probabilities, key=lambda x: x[1], reverse=True)[:top_n]

       
        topic_info = {"content": content_list[idx]}
        for rank, (topic_id, prob) in enumerate(sorted_topics):
            topic_info[f"Chủ đề {rank + 1}"] = f"Chủ đề {topic_id} ({prob * 100:.2f}%)"
        
        
        result.append(topic_info)
    
 
    return pd.DataFrame(result)


content_list = df['content'].tolist()


top_n_topics = 3


topic_distribution_df = get_topic_distribution_for_documents(
    lda_model, corpus, content_list, top_n=top_n_topics
)


output_file = "/kaggle/working/phân_phối_của_chủ dề_Giáo_dục.csv"
topic_distribution_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"File đã được lưu tại: {output_file}")


# Diễn giải

In [ ]:
import pandas as pd

file_name = "/kaggle/working/phân_phối_của_chủ dề_Giáo_dục.csv"
df = pd.read_csv(file_name, encoding="utf-8-sig")

columns_to_drop = df.columns[4:]
df.drop(columns=columns_to_drop, inplace=True)
print(df)

In [ ]:
import pandas as pd

output_file_name = "/kaggle/working/diễn_giải_Giáo_dục.csv"

labels_map = {
    "Chủ đề 0": "Phổ thông",
    "Chủ đề 1": "Đại học",
    "Chủ đề 2": "Tuyển sinh",
}

df['Chủ đề 1'] = df['Chủ đề 1'].str.extract(r"(Chủ đề \d+)")[0]
df['Chủ đề 1'] = df['Chủ đề 1'].map(labels_map)
df.rename(columns={"Chủ đề 1": "Chủ đề chính"}, inplace=True)

df.to_csv(output_file_name, index=False, encoding="utf-8-sig")

In [ ]:
output_folder = "/kaggle/working/diễn_giải_Giáo_dục"
df_grouped = df.groupby("Chủ đề chính")

print(df_grouped)

file_paths = {}
for group_name, group_data in df_grouped:
    file_name = f"{output_folder}/{group_name.replace(' ', '_')}.csv"
    group_data.to_csv(file_name, index=False, encoding="utf-8-sig")
    file_paths[group_name] = file_name

print("Các file đã được lưu:", file_paths)